# Install package ที่จำเป็นต้องใช้

In [1]:
!pip install selenium #selenium
!pip install bs4 #beautiful soup

# Import function ที่จำเป็น

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
import bs4
import time
import re

# สร้าง Web driver ของ Firefox
โดยการเช็ค Version Driver ก่อน Dowload 
https://github.com/mozilla/geckodriver/releases

เพื่อใช้ในการเปิด Website ที่ต้องการ Scraping ( Selenium เหมาะกับการใช้เปิด website ประเภท Dynamic Website )


In [3]:
options = Options()
options.binary_location = r'/Applications/Firefox.app/Contents/MacOS/firefox-bin'
driver = webdriver.Firefox(executable_path=r'/Users/peakeiei/dsi310_scrapy/geckodriver', options=options)
driver.get('https://shopee.co.th/search?brands=2240042&keyword=%E0%B8%82%E0%B8%99%E0%B8%A1%E0%B9%80%E0%B8%88%E0%B8%A5%E0%B8%A5%E0%B8%B5%E0%B9%88%20haribo&noCorrection=t')

<ipython-input-3-76c4c9b54f0d>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=r'/Users/peakeiei/dsi310_scrapy/geckodriver', options=options)


เรียกใช้ Selenium และเปิดเว็ปที่ต้องการ scrap ซึ่ง website ที่ใช้ Shopee จะมีหน้าในการเลือกภาษาและ Zoomout page เพื่อโหลดข้อมูล 
( ขั้นตอนนี้ขึ้นอยู่แต่ละ website ที่จะนำไป scrap ) 

In [4]:
#เลือกภาษาไทย Selenium
thai_button = driver.find_element("xpath",'/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button').click()
time.sleep(6)

In [5]:
#Zoom out เพื่อให้ load ทั่วหน้าจอ
driver.execute_script("document.body.style.MozTransform='scale(0.1)';")
driver.execute_script('document.body.style.MozTransformOrigin = "0 0";')

# เริ่มต้นขั้นตอน Scraping โดยใช้ BS4

In [6]:
data = driver.page_source #ดึงข้อมูลจากหน้าเว็บ
soup = bs4.BeautifulSoup(data) #จัดในรูปแบบ BeautifulSoup
time.sleep(6)

In [7]:
soup.select(".row>div") #css selector

[<div class="col-xs-2-4 shopee-search-item-result__item" data-sqe="item"><a data-sqe="link" href="/Haribo-Jelly-Strawberry-200g-ฮาริโบ้-เจลลี่-สตรอเบอร์รี่-i.16354168.17388312451?sp_atk=f4ee3d13-faf3-45c6-b610-306d399081bc&amp;xptdk=f4ee3d13-faf3-45c6-b610-306d399081bc"><div class="tWpFe2"><div class="VTjd7p whIxGK"><div style="pointer-events: none;"><div class="yvbeD6 KUUypF"><img alt="Haribo Jelly Strawberry 200g ฮาริโบ้ เจลลี่ สตรอเบอร์รี่" class="_7DTxhh vc8g9F" height="invalid-value" src="https://cf.shopee.co.th/file/sg-11134201-22100-6z1t7k0rj2ivab_tn" style="object-fit: contain" width="invalid-value"/><div class="IpGwg7"><div class="customized-overlay-image"><img height="" src="https://cf.shopee.co.th/file/ad43aca11fa2eddf2933a343ea753ffc" width=""/></div></div></div></div><div class="KMyn8J"><div class="dpiR4u" data-sqe="name"><div class="FDn--+"><div class="ie3A+n bM+7UW Cve6sh">Haribo Jelly Strawberry 200g ฮาริโบ้ เจลลี่ สตรอเบอร์รี่</div></div></div><div class="hpDKMN"><div 

In [8]:
el=soup.select(".row>div")

In [9]:
len(el)

46

ทดลอง Scrap ชื่อสินค้า

In [10]:
e = el[0] #แทนกล่องของสินค้าแต่ละกล่อง

In [11]:
#ดึงข้อมูลชื่อ
e.select_one("div.col-xs-2-4:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)").text.strip()

'Haribo Jelly Strawberry 200g ฮาริโบ้ เจลลี่ สตรอเบอร์รี่'

ทอลอง Scrap ราคา

In [12]:
#ราคา A baht - B baht, ราคาเต็ม
e.select_one("div.col-xs-2-4:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2)").text.strip()

'฿99'

In [13]:
#รวม ราคาถูกสุด
try:
    p = e.select_one("div.col-xs-2-4:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2)").text.strip()
except:
    p = e.select_one("div.col-xs-2-4:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2)").text.strip()
p = p.replace(",", "").replace("฿", "").replace(" ", "")
try:
    start_price = float(p)
except:
    price = p.rpartition('-')[0]
    start_price = float(price)

start_price

99.0

ทดลอง Scrap ยอดขาย


In [14]:
#Scrape ยอดขาย
sales = e.select_one("div.col-xs-2-4:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(3) > div:nth-child(2)").text.strip().replace("พัน","000").replace("ล้าน","000000")
try:
    sales = float(re.sub('\D','',sales))
except:
    sales = 0
sales

0

ทดลอง Scrap จังหวัด

In [15]:
#Scrape จังหวัด
#จังหวัด
e.select_one("div.col-xs-2-4:nth-child(1) > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(4)").text.strip()

'จังหวัดกรุงเทพมหานคร'

## นำขั้นตอนการเก็บข้อมูลทั้งหมดมาเข้า Loop เพื่อทำการ Scrap สินค้าทั้งหมดในทุกหน้า

In [16]:
time.sleep(6)
records=[]
for i in range(1):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    el=soup.select(".row>div")
    for e in el:
        name = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text.strip()
        try:
            p = e.select_one("div.col-xs-2-4 > a > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(2)").text.strip()
            if p == '':
                print(1/0)
        except:
            p = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)").text.strip()
        p = p.replace(",", "").replace("฿", "").replace(" ", "")
        try:
            start_price = float(p)
        except:
            price = p.rpartition('-')[0]
            start_price = float(price)

        sales = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(3)").text.strip().replace("พัน","000").replace("ล้าน","000000")
        try:
            sales = float(re.sub('\D','',sales))
        except:
            sales = 0
        province = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(4)").text.strip()
        #print(name, p)
        records.append([name, start_price,sales,province])

In [17]:
len(records)

46

## นำข้อมูลที่ได้ใส่ลงใน DataFrame

In [18]:
#ทำเป็น DataFrame
df = pd.DataFrame(records, columns=['itemname','price range','sales','province'])
df

,itemname,price range,sales,province
0,Haribo Jelly Strawberry 200g ฮาริโบ้ เจลลี่ สต...,99.0,0.0,จังหวัดกรุงเทพมหานคร
1,Haribo Pico Balla 175g ฮาริโบ้ เยลลี่ หรือกัมม...,96.0,2.0,จังหวัดกรุงเทพมหานคร
2,Haribo Berries Jelly 200g เยลลี่ หรือกัมมี่แบ...,96.0,7.0,จังหวัดกรุงเทพมหานคร
3,Haribo Salino Jelly 200g เยลลี่ หรือกัมมี่แบร์...,96.0,3.0,จังหวัดกรุงเทพมหานคร
4,Haribo Happy Grapes 80g รสองุ่น เยลลี่ หรือกัม...,48.0,6.0,จังหวัดกรุงเทพมหานคร
5,Haribo Goldbears Happy Cola Happy Mix Berries ...,68.0,84000.0,จังหวัดกรุงเทพมหานคร
6,Haribo กัมมี่ เจลลี่ ฮาริโบ้ ขนาด 80 กรัม (เลื...,34.0,53000.0,จังหวัดสมุทรปราการ
7,Haribo Milchbaran Haribo 175g เยลลี่ หรือกัมมี...,105.0,0.0,จังหวัดกรุงเทพมหานคร
8,Haribo Konfekt Jelly 200g เยลลี่ หรือกัมมี่แบร...,96.0,0.0,จังหวัดกรุงเทพมหานคร
9,Haribo Goldbears Happy Mix Berries กัมมี่ เจลล...,100.0,0.0,จังหวัดกรุงเทพมหานคร


แปลงเป็นไฟล์ Excel บันทึกอักษรเป็นแบบ มาตรฐาน UTF-8

In [19]:
df.to_excel('Shopee_Haribo.xlsx', encoding='utf-8')